# Работа со строковыми значениями

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [132]:
import pandas as pd
df = pd.read_csv('recipes_sample.csv')
df.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [126]:
df = pd.read_csv('recipes_sample.csv')
id_max_len = len(str(df.id.max()))
min_max_len = len(str(df.minutes.max()))
print('|'+' '*(int(id_max_len/2+3))+'id'+ ' '*(int(id_max_len/2+3))+'|' +' '*(int(min_max_len/2))+'minutes'+ ' '*(int(min_max_len/2))+'|')
len__ = len('|'+' '*(int(id_max_len/2+3))+'id'+ ' '*(int(id_max_len/2+3))+'|' +' '*(int(min_max_len/2))+'minutes'+ ' '*(int(min_max_len/2))+'|')
print('|'+'-'*(len__-2)+'|')
for i in range(len(df.sample(5))):
    l = df.sample(5)['id'].iloc[i]
    lm = df.sample(5)['minutes'].iloc[i] 
    print('|'+' '*4+str(l)+' '*(id_max_len - len(str(l))+4)+'|' +' '*5+str(lm)+' '*(min_max_len- len(str(lm))+2)+'|')

|      id      |   minutes   |
|----------------------------|
|    325557    |     45      |
|    328567    |     65      |
|    230483    |     75      |
|    66519     |     110     |
|    410482    |     25      |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1.
2. Шаг 2.
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [221]:
def show_info(name, steps, minutes, author_id):
    string = f'"{name.title()}"\n\n'+'\n'.join([str(steps.index(i)+1)+'. '+ i.capitalize() for i in steps])+'\n'+'-'*(max([len(el) for el in steps])+3)+'\n'+f'Автор: {author_id}\nСреднее время приготовления: {minutes} минут'  
    return string

print(show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    ))

"George S At The Cove Black Bean Soup"

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
------------------------------------------------------
Автор: 35193
Среднее время приготовления: 90 минут


Считаю, что количество символов '-' лучше будет привести к универсальному виду, поэтому предоставляю два варианта. Ниже тото - который подходит для данного assert, выше - тот, который я посчитал улучшенным

In [223]:
def show_info(name, steps, minutes, author_id):
    string = f'"{name.title()}"\n\n'+'\n'.join([str(steps.index(i)+1)+'. '+ i.capitalize() for i in steps])+'\n----------\n'+f'Автор: {author_id}\nСреднее время приготовления: {minutes} минут\n'  
    return string


assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [230]:
from bs4 import BeautifulSoup as soup
df1 = pd.read_csv('recipes_sample.csv', index_col = 1)
with open(
    'steps_sample.xml',
    'r',
    encoding='utf-8'
) as f:
    steps_file = soup(f)
    
recipes = steps_file.find_all('recipe')
steps = {}
for recipe in recipes:
    steps[recipe.find('id').text] = [steps.text for steps in recipe.find_all('step')]

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [279]:
steps
import re
pattern = re.findall('\d+ minutes|hours', 'in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes 10 hours')
pattern
listt_ = []
for el in steps['25082']:
    listt_.append(re.findall('\d+\s(?:hours|hour|minutes|minute)', el))

list(filter(lambda x: x != [], listt_))

[['20 minutes'],
 ['10 minutes'],
 ['2 hours'],
 ['10 minutes'],
 ['20 minutes', '30 minutes']]

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
